In [29]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, models
import random
import os, pathlib
from PIL import Image
import matplotlib.pyplot as plt

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
DATA_DIR = 'drive/MyDrive/term_project/%s'

NUM_CHANNELS = 3
WIDTH = HEIGHT = 128
NUM_CLASSES = len(os.listdir(DATA_DIR % 'annotations'))

In [32]:
N_EPOCHS = 50
BATCH_SIZE_TRAIN = 50
BATCH_SIZE_TEST = 50
LR = 0.001

In [33]:
def get_labels(labels_dir, images_dir):
    label_names = [os.path.splitext(f)[0] for f in os.listdir(labels_dir)]
    image_indexes = [os.path.splitext(f)[0] for f in os.listdir(images_dir)]

    label_indexes = {file_name: set() for file_name in label_names}
    for file_name in label_names:
        with open(os.path.join(labels_dir, file_name + '.txt')) as f:
            for line in f: label_indexes[file_name].add(line.strip()) 

    image_labels = {index: [] for index in image_indexes}
    for label in label_names:
        for key in image_labels.keys():
            image_labels[key].append(int(str(key[2:]) in label_indexes[label]))

    return label_names, image_labels

In [34]:
from torch.utils.data import Dataset
from PIL import Image

class ImageDataset(Dataset):
    
    def __init__(self, image_dir: str, label_dir: str, transform = None) -> None:
        self.paths = [os.path.join(image_dir, file_name) for file_name in os.listdir(image_dir)]
        self.transform = transform
        self.labels, self.idx_to_label = get_labels(label_dir, image_dir)

    def load_image(self, index: int) -> Image.Image:
        "Opens an image via a path and returns it."
        image_path = self.paths[index]
        return Image.open(image_path).convert('RGB')
    
    def __len__(self) -> int:
        "Returns the total number of samples."
        return len(self.paths)
    
    def __getitem__(self, index: int):
        "Returns one sample of data, data and label (X, y)."
        image = self.load_image(index)
        image_name = os.path.splitext(os.path.basename(self.paths[index]))[0]
        label = torch.Tensor(self.idx_to_label[image_name])

        if self.transform:
            return self.transform(image), label 
        else:
            return image, label 

In [35]:
dataset = ImageDataset(DATA_DIR % 'images', DATA_DIR % 'annotations')

test_len = val_len = int(len(dataset) * 0.05) 
train_len = len(dataset) - test_len - val_len

train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len + val_len])
val_set, test_set = torch.utils.data.random_split(test_set, [val_len, test_len])

In [36]:
train_transform = transforms.Compose([
                                    transforms.RandomResizedCrop(128),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                    ])

test_transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                    ])

test_set.dataset.transform = test_transform
val_set.dataset.transform = test_transform
train_set.dataset.transform = train_transform

In [37]:
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE_TEST, shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset=val_set, shuffle=False)

In [38]:
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(2048, NUM_CLASSES)

loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)


In [39]:
dev_loss = np.iinfo(int).max

def validation_loss(model, dev_loader):
    total_correct = 0
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for (data, target) in dev_loader:

            output = model.forward(data)
            total_loss += loss_function(output, target)
            predicted = torch.round(torch.sigmoid(output))
            total_correct += torch.sum(predicted == target) / NUM_CLASSES

    return total_loss, total_correct

for epoch in range(N_EPOCHS):
    train_loss = 0
    train_correct = 0

    total = 0
    model.train()
    for batch_num, (data, target) in enumerate(train_loader):

        optimizer.zero_grad()
        output = model.forward(data)
        loss = loss_function(output, target)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

        total += BATCH_SIZE_TRAIN
        predicted = torch.round(torch.sigmoid(output))
        train_correct += torch.sum(predicted == target) / NUM_CLASSES

    print('Training: Epoch %d - Loss: %.4f | Train Acc: %.3f%% (%d/%d)' % 
        (epoch, train_loss / len(train_loader), 
        100. * train_correct / total, train_correct, total))

    dev_loss_current, dev_correct = validation_loss(model, val_loader)
    print('Validation: Loss: %.4f | Acc: %.3f%% (%d/%d)' % (dev_loss_current / len(val_loader), 
        100. * dev_correct / len(val_loader), dev_correct, len(val_loader)))

    # if (dev_loss_current > dev_loss):
    #     print("Early Stopping: Epoch %d" % epoch)
    #     break
    # else: dev_loss = dev_loss_current

Training: Epoch 0 - Loss: 0.1422 | Train Acc: 94.941% (15522/16350)
Validation: Loss: 0.1188 | Acc: 95.739% (868/907)
Training: Epoch 1 - Loss: 0.1178 | Train Acc: 95.504% (15614/16350)
Validation: Loss: 0.1100 | Acc: 95.881% (869/907)
Training: Epoch 2 - Loss: 0.1155 | Train Acc: 95.581% (15627/16350)
Validation: Loss: 0.1110 | Acc: 95.881% (869/907)
Training: Epoch 3 - Loss: 0.1129 | Train Acc: 95.627% (15635/16350)
Validation: Loss: 0.1067 | Acc: 96.039% (871/907)
Training: Epoch 4 - Loss: 0.1126 | Train Acc: 95.683% (15644/16350)
Validation: Loss: 0.1075 | Acc: 95.991% (870/907)
Training: Epoch 5 - Loss: 0.1122 | Train Acc: 95.664% (15641/16350)
Validation: Loss: 0.1110 | Acc: 95.865% (869/907)


KeyboardInterrupt: ignored